In [1]:
!pip install tensorflow

In [2]:
from tensorflow.keras.applications import Xception  
from tensorflow.keras.models import Model  
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D  
from tensorflow.keras.preprocessing.image import ImageDataGenerator  

base_model = Xception(weights='imagenet', include_top=False)  
x = base_model.output  
x = GlobalAveragePooling2D()(x)  
x = Dense(1024, activation='relu')(x)  
predictions = Dense(2, activation='softmax')(x) 

model = Model(inputs=base_model.input, outputs=predictions)  

# Freeze the base model layers  
for layer in base_model.layers:  
    layer.trainable = False

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  
train_generator = train_datagen.flow_from_directory(  
    '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train',  # Path to your training data  
    target_size=(299, 299),  
    batch_size=32,  
    class_mode='categorical',  
    subset='training')  
validation_generator = train_datagen.flow_from_directory(  
    '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/train',  
    target_size=(299, 299),  
    batch_size=32,  
    class_mode='categorical',  
    subset='validation')

Found 80000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [5]:
model.fit(train_generator, validation_data=validation_generator, epochs=10)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2500/2500 ━━━━━━━━━━━━━━━━━━━━ 740s 290ms/step - accuracy: 0.8624 - loss: 0.3156 - val_accuracy: 0.9098 - val_loss: 0.2186
Epoch 2/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 388s 155ms/step - accuracy: 0.9183 - loss: 0.1995 - val_accuracy: 0.9287 - val_loss: 0.1815
Epoch 3/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 382s 153ms/step - accuracy: 0.9345 - loss: 0.1622 - val_accuracy: 0.9272 - val_loss: 0.1847
Epoch 4/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 377s 151ms/step - accuracy: 0.9499 - loss: 0.1272 - val_accuracy: 0.9261 - val_loss: 0.1901
Epoch 5/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 378s 151ms/step - accuracy: 0.9590 - loss: 0.1049 - val_accuracy: 0.9310 - val_loss: 0.1901
Epoch 6/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 378s 151ms/step - accuracy: 0.9680 - loss: 0.0846 - val_accuracy: 0.9277 - val_loss: 0.1999
Epoch 7/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 378s 151ms/step - accuracy: 0.9735 - loss: 0.0697 - val_accuracy: 0.9291 - val_loss: 0.2092
Epoch 8/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 380s 152ms/step - accuracy: 0.9

In [6]:
model.save('xception_model.h5')

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator  

# Create a test data generator  
test_datagen = ImageDataGenerator(rescale=1./255)  

test_generator = test_datagen.flow_from_directory(  
    '/kaggle/input/cifake-real-and-ai-generated-synthetic-images/test',  # Path to your test dataset  
    target_size=(299, 299),  # Resize the images to match the input size  
    batch_size=32,  
    class_mode='categorical',  # Set to 'categorical' for multi-class  
    shuffle=False  # Important: Do not shuffle the test data  
)

Found 20000 images belonging to 2 classes.


In [8]:
# Evaluate the model on the test data  
test_loss, test_accuracy = model.evaluate(test_generator)  

print(f'Test Loss: {test_loss}')  
print(f'Test Accuracy: {test_accuracy}')

625/625 ━━━━━━━━━━━━━━━━━━━━ 148s 236ms/step - accuracy: 0.9324 - loss: 0.2640
Test Loss: 0.27196863293647766
Test Accuracy: 0.9302999973297119
